In [1]:
# Common Setup

from torchvision.datasets import MNIST
from torchvision import transforms

import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

def plot_embeddings(embeddings, targets, class_labels, title=None, xlim=None, ylim=None):
    plt.figure(figsize=(5,5))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    if title:
        plt.title(title)
    plt.legend(class_labels, loc='upper right')

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 2))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

In [2]:
# MNSIST dataset setup

mean, std = 0.1307, 0.3081
n_classes = 10
class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

train_dataset = MNIST('MNIST', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
test_dataset = MNIST('MNIST', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                            ]))

n_classes = 10
n_samples = 25

In [3]:
# Blindfolded setup
from torch.utils.data import DataLoader
from blindflded_triplets_dataset import BlindfoldedTripletsDataset

batch_size = 2
sensor_key = 'accel_y' # selected for visualization
sensor_keys = ['accel_x', 'accel_y', 'accel_z', 'rot_h', 'rot_p', 'rot_r']

Blindfolded_Triplets_Dataset_train = BlindfoldedTripletsDataset(filepath = '../Places/PopulatedPlaces.json', sensor_keys=sensor_keys, n_selection=7, train=True)
Blindfolded_Triplets_Dataset_test = BlindfoldedTripletsDataset(filepath = '../Places/PopulatedPlaces.json', sensor_keys=sensor_keys, n_selection=7, train=False)

# sensor_idx = Blindfolded_Triplets_Dataset_train.tensor_index_translation(sensor_key)
n_classes = int(max(Blindfolded_Triplets_Dataset_train.labels)) + 1
n_samples = 6

train_dataset = Blindfolded_Triplets_Dataset_train
# test_dataset = Blindfolded_Triplets_Dataset_test

n_samples = 6
n_classes = int(max(train_dataset.labels)) + 1
train_dataset = Blindfolded_Triplets_Dataset_train
test_dataset = Blindfolded_Triplets_Dataset_test

basic_dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
basic_dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

load_places_from_json(): loading .json file
load_places_from_json(): loaded 208 items
load_places_from_json(): loading .json file
load_places_from_json(): loaded 208 items


In [4]:
import torch
import torch.nn as nn

class BlindfoldedMLP(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(BlindfoldedMLP, self).__init__()
        self.input_dim = 6 * 2400  # Fixed input size
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.fc1 = nn.Linear(self.input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Ensure x is properly reshaped
        x = x.view(x.shape[0], -1)  # Flatten (batch_size, 6, 2400) -> (batch_size, 14400)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def get_embedding(self, x):
        return self.forward(x)

In [18]:

test_series = train_dataset.__getitem__(0)

# print(train_dataset.data)
# print(test_series)

print(test_series[0].shape)
# for attr in dir(train_dataset)[:]:
#     print(f"{attr}: {type(getattr(train_dataset, attr))}")



torch.Size([1, 6, 2400])


In [5]:
# Set up the network and training parameters
from networks import EmbeddingNet
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric
from datasets import BalancedBatchSampler

# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(train_dataset.train_labels, n_classes=n_classes, n_samples=n_samples)
test_batch_sampler = BalancedBatchSampler(test_dataset.test_labels, n_classes=10, n_samples=n_samples)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)


# for (data, target), batch_idx in enumerate(online_train_loader):
#     # batch_idx is the index of the current batch
#     # data is the input data for the current batch
#     # target is the target labels for the current batch
#     print(f'Batch index: {batch_idx}')
#     print(f'Data shape: {data.shape}')
#     print(f'Target shape: {target.shape}')

margin = 1.
embedding_net = EmbeddingNet()
blindfolded_MLP = BlindfoldedMLP(hidden_dim=128, output_dim=64)
model = blindfolded_MLP
if cuda:
    print("cuda")
    model.cuda()
print(next(model.parameters()).device)
loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 2
log_interval = 50

# Training
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])
# fit(basic_dataloader_train, basic_dataloader_test, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

# Plotting
train_embeddings_otl, train_labels_otl = extract_embeddings(online_train_loader, model)
plot_embeddings(train_embeddings_otl, train_labels_otl, title="Training set", class_labels=class_labels)
val_embeddings_otl, val_labels_otl = extract_embeddings(online_test_loader, model)
plot_embeddings(val_embeddings_otl, val_labels_otl, title="Training set", class_labels=class_labels)
plt.show()

cpu


/Users/phkunze/opt/anaconda3/envs/going_blindfolded/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


UnboundLocalError: cannot access local variable 'batch_idx' where it is not associated with a value